# Building the pre-processing page

For the pre-processing page, I have included several tabs that correspond to various pre-processing steps.  

    1. Tokenization
    2. Removal of stop words
    3. Stemming words
    4. Work embedding
    

When you click "continue" all the options that were set on each tab will be used to process the data.

The first thing to realize when creating a panel pipeline is that each page must be built out of a paramaterized class.  It will need to include a `panel` method, which specifies how the pane should display.  It should also include an `output` function with the `param.output` decorator with variables to be passed to the next stage.  Here I am going to create my first stage by making a class called `PreProcessor`.  I will make sure to include the output and panel functions and will fill in with other needed parts to make my stage.

I am going to quickly go through my process to build this first stage, but will not explain everything in detail.  Please see panel docs for more information on setting up simple panel app.

Here is the barebones start of my `PreProcessor` class.  I have included an `output` and a `panel` function.  We will fill these in as we go along.

In [ ]:
import panel as pn
import param

class PreProcessor(param.Parameterized):
    
    
    def __init__(self, **params):
        super().__init__(**params)
        
        
    @param.output()
    def output(self):
        return #output
    
   
    
    def panel(self):
        
        return #panel set up
        

I am going to begin filling in with more pieces by building the tabs.  First, I want to set the parts in place to display these tabs.  There is also a continue button included on the page, so I will put this just below the tabs.

In [ ]:
import panel as pn
import param

class PreProcessor(param.Parameterized):
    
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.continue_button = pn.widgets.Button(name='Continue',
                                                 width = 100,
                                                 button_type='primary')
        
    @param.output()
    def output(self):
        return #output

    
    def panel(self):
        
        return return pn.Column(
            pn.Tabs(
                    #tabs to be added.
                ),
            self.continue_button
        )
        

Now that I am ready to accept tabs, let's create a tab to include on the page.  I decided that each tab would have roughly the same layout.  It would include a left pane and right pane.  The left pane would include the name of the pre-processing step, some explanatory text, then all the widgets needed to insert information for this step (buttons, checkboxes, input text, etc).  The right side will be a display pane for the uploaded text, and eventually the final results.  

I'll start by including the first tab  that loads text.  I'll simply create another function called `load_text` that can be inserted into the `tabs` portion of the `panel` function.  The `load_text_page` function is basically like a `panel` function in that it will return a panel layout.  `load_text_page` includes a file input button that will need to be uploaded properly, which will be done with `load_df`.  

Furthermore, the right side will include a display of the dataframe once loaded.  I have created a function, `df_pane`, that will display this dataframe, `display_df`, when it is present.  `df_pane` will be used on any page you choose to display a dataframe.  Currently, I have it set to display only on the load text tab.

In [ ]:
import panel as pn
import param

class PreProcessor(param.Parameterized):
    
    df = param.DataFrame()
    
    display_df = param.DataFrame(default = pd.DataFrame())
    
    def __init__(self, **params):
        super().__init__(**params)
        
        # button for the pre-processing page
        self.continue_button = pn.widgets.Button(name='Continue',
                                                 width = 100,
                                                 button_type='primary')
        
        # load text widgets 
        self.header_checkbox = pn.widgets.Checkbox(name='Header included in file')
        self.load_file = pn.widgets.FileInput()
        self.load_file.link(self.df, callbacks={'value': self.load_df})

        
    @param.output()
    def output(self):
        return #output
    
    
    @param.depends('display_df')
    def df_pane(self):
        return pn.WidgetBox(self.display_df,
                           height = 300,
                           width = 400)
    
    
    # load text page functions
    #-----------------------------------------------------------------------------------------------------
    def load_df(self, df, event):
        info = io.BytesIO(self.load_file.value)
        if self.header_checkbox.value==True:
            self.df = pd.read_csv(info)
        else:
            self.df = pd.read_csv(info, sep='\n', header = None, names=['text'])
        
        self.display_df = self.df
    
    def load_text_page(self):
        helper_text = (
            "This simple Sentiment Analysis NLP app will allow you to select a few different options " +
            "for some preprocessing steps to prepare your text for testing and training. " +
            "It will then allow you to choose a model to train, the percentage of data to " +
            "preserve for test, while the rest will be used to train the model.  Finally, " +
            "some initial metrics will be displayed to determine how well the model did to predict " +
            "the testing results." +
            " " +
            "Please choose a csv file that contains lines of text to analyze.  This text should " +
            "have a text column as well as a sentiment column.  If there is a header included in the file, " +
            "make sure to check the header checkbox."
        )
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Load Text:'),
                    pn.Column(
                        helper_text,
                         self.header_checkbox,
                         self.load_file
                        ),
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    self.df_pane,
                    
                )
        
        )

    #-----------------------------------------------------------------------------------------------------
    
    
    def panel(self):
        
        return pn.Column(
            pn.Tabs(
                ('Load Text', self.load_text_page),
                ),
            self.continue_button
        )
        

At this point, we can run the following to see how our panel app is coming together.  

In [ ]:
pp = PreProcessor()
pp.panel()

We have one tab in place; now, I will go ahead and add the other functions, widgets and signals needed for each of the other tabs.  They will all be done in a similar fashion to the load text tab.


In [ ]:
import panel as pn
import param
import pandas as pd

class PreProcessor(param.Parameterized):
    
    #df will be the variable holding the dataframe of text
    df = param.DataFrame()
    #title to display for each tab
    name_of_page = param.String(default = 'Name of page')
    #dataframe to display.
    display_df = param.DataFrame(default = pd.DataFrame())
    # stopword_df is the dataframe containing the stopewords
    stopword_df = param.DataFrame(default = pd.DataFrame())
    
    stopwords = param.List(default = [])
    X = param.Array(default = None)
    
    
    def __init__(self, **params):
        super().__init__(**params)
        
        
        
        # button for the pre-processing page
        self.continue_button = pn.widgets.Button(name='Continue',
                                                 width = 100,
                                                 button_type='primary')
        
        # load text widgets 
        self.header_checkbox = pn.widgets.Checkbox(name='Header included in file')
        self.load_file = pn.widgets.FileInput()
        self.load_file.link(self.df, callbacks={'value': self.load_df})
        self.header_checkbox = pn.widgets.Checkbox(name='Header included in file')
        
        # tokenize widgets
        self.search_pattern_input = pn.widgets.TextInput(name='Search Pattern', value = '\w+', width = 100)
        
        # remove stop words widgets
        self.load_words_button = pn.widgets.FileInput()
        self.load_words_button.link(self.stopwords, callbacks={'value': self.load_stopwords})
        
        # stem widgets
        self.stem_choice = pn.widgets.Select(name='Select', options=['Porter', 'Snowball'])
        
        # embedding widgets
        
        self.we_model = pn.widgets.Select(name='Select', options=['SKLearn Count Vectorizer'])

        
    @param.output()
    def output(self):
        return None
    
    
    @param.depends('display_df')
    def df_pane(self):
        return pn.WidgetBox(self.display_df,
                           height = 300,
                           width = 400)
    
    # load text page functions
    #-----------------------------------------------------------------------------------------------------
    def load_df(self, df, event):
        info = io.BytesIO(self.load_file.value)
        if self.header_checkbox.value==True:
            self.df = pd.read_csv(info)
        else:
            self.df = pd.read_csv(info, sep='\n', header = None, names=['text'])
        
        self.display_df = self.df
    
    def load_text_page(self):
        helper_text = (
            "This simple Sentiment Analysis NLP app will allow you to select a few different options " +
            "for some preprocessing steps to prepare your text for testing and training. " +
            "It will then allow you to choose a model to train, the percentage of data to " +
            "preserve for test, while the rest will be used to train the model.  Finally, " +
            "some initial metrics will be displayed to determine how well the model did to predict " +
            "the testing results." +
            " " +
            "Please choose a csv file that contains lines of text to analyze.  This text should " +
            "have a text column as well as a sentiment column.  If there is a header included in the file, " +
            "make sure to check the header checkbox."
        )
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Load Text:'),
                    pn.Column(
                        helper_text,
                         self.header_checkbox,
                         self.load_file
                        ),
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    self.df_pane,
                    
                )
        
        )

    #-----------------------------------------------------------------------------------------------------
    
    # tokenize page options
    #-----------------------------------------------------------------------------------------------------
    def tokenize_option_page(self):
        
        help_text = ("Tokenization will break your text into a list of single articles " +
            "(ex. ['A', 'cat', 'walked', 'into', 'the', 'house', '.']).  Specify a regular " +
            "expression (regex) search pattern to use for splitting the text.")
        
        return pn.Column(
                    pn.pane.Markdown(f'##Tokenize options:'),
                    pn.WidgetBox(help_text, self.search_pattern_input,
                                    height = 300,
                                    width = 300
        
                                )
                )
    
    #-----------------------------------------------------------------------------------------------------
    
    
    # remove stopwords page 
    #-----------------------------------------------------------------------------------------------------
    
    def remove_stopwords_page(self):
        
        help_text = (
            "Stop words are words that do not add any value to the sentiment of the text. " +
            "Removing them may improve your sentiment results.  You may load a list of stop words " +
            "to exclude from your text."
        )
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Load Stopwords:'),
                    pn.WidgetBox(help_text, self.load_words_button,
                                    height = 300,
                                    width = 300
        
                    )
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    pn.WidgetBox(self.stopword_df,
                           height = 300,
                           width = 400)
                    
                )
        )
    
    def load_stopwords(self, stopwords, event):
        info = io.BytesIO(self.load_words_button.value)
        self.stopwords = pd.read_pickle(info)
        self.stopword_df = pd.DataFrame({'stop words': self.stopwords})

    #-----------------------------------------------------------------------------------------------------
    
    # stemming page 
    #-----------------------------------------------------------------------------------------------------
    
    def stemmer_page(self):
        help_text = (
            "Stemming is a normalization step for the words in your text.  Something that is " +
            "plural should probably still be clumped together with a singular version of a word, " +
            "for example.  Stemming will basically remove the ends of words.  Here you can choose " + 
            "between a Porter Stemmer or Snowball Stemmer. Porter is a little less aggressive than " +
            "Snowball, however, Snowball is considered a slight improvement over Porter."
        )
        return pn.Column(
                    pn.pane.Markdown(f'##Stemmer options:'),
                    pn.WidgetBox(help_text, self.stem_choice,
                height = 300,
                width = 300)
                )
    
    #-----------------------------------------------------------------------------------------------------
    
    # embedding page 
    #-----------------------------------------------------------------------------------------------------
    
    def word_embedding_page(self):
        
        help_text = ("Embedding the process of turning words into numerical vectors. " +
                    "There have been several algorithms developed to do this, however, currently in this " +
                    "app, the sklearn count vectorizer is available. This algorithm will return a sparse " +
                    "matrix represention of all the words in your text."
                    )
        
        
        
        return pn.Column(
                    pn.pane.Markdown(f'##Choose embedding model:'),
                    pn.WidgetBox(help_text, self.we_model,
                            height = 300,
                            width = 300
        
                    )
        
                )
    
    #-----------------------------------------------------------------------------------------------------
    
        
        
    
    def panel(self):
        
        return pn.Column(
            pn.Tabs(
                ('Load Text', self.load_text_page),
                ('Tokenize', self.tokenize_option_page),
                ('Remove Stopwords', self.remove_stopwords_page),
                ('Stem', self.stemmer_page),
                ('Embed', self.word_embedding_page)
                ),
            self.continue_button
        )
        

Now let's look at our panel app by running the cell below. As you can see, it is starting to come together.  This is pretty much the first stage!  

In [ ]:
pp = PreProcessor()
pp.panel()

The only thing we don't have is the actual processing being done on the data.  We can set up all our choices for each tab, but we haven't done anything with it yet.  This is where the `continue` button will come into play.  With the `continue` button, we can run our processing, and send a signal that its time to move on to the next page.  I will explain more about moving onto the next page in a moment. Let's first add a function that will put all of our pre-processing pieces together.  The `continue_ready` function will be added to our PreProcessor class, and the continue button will be told to run this function when it is clicked.  

In [ ]:
import panel as pn
import param
import pandas as pd

class PreProcessor(param.Parameterized):
    
    #df will be the variable holding the dataframe of text
    df = param.DataFrame()
    #title to display for each tab
    name_of_page = param.String(default = 'Name of page')
    #dataframe to display.
    display_df = param.DataFrame(default = pd.DataFrame())
    # stopword_df is the dataframe containing the stopewords
    stopword_df = param.DataFrame(default = pd.DataFrame())
    
    stopwords = param.List(default = [])
    X = param.Array(default = None)
    
    
    def __init__(self, **params):
        super().__init__(**params)
        
        
        
        # button for the pre-processing page
        self.continue_button = pn.widgets.Button(name='Continue',
                                                 width = 100,
                                                 button_type='primary')
        # ****NEW ADDITION******
        self.continue_button.on_click(self.continue_ready)
        # **********************
        
        # load text widgets 
        self.header_checkbox = pn.widgets.Checkbox(name='Header included in file')
        self.load_file = pn.widgets.FileInput()
        self.load_file.link(self.df, callbacks={'value': self.load_df})
        self.header_checkbox = pn.widgets.Checkbox(name='Header included in file')
        
        # tokenize widgets
        self.search_pattern_input = pn.widgets.TextInput(name='Search Pattern', value = '\w+', width = 100)
        
        # remove stop words widgets
        self.load_words_button = pn.widgets.FileInput()
        self.load_words_button.link(self.stopwords, callbacks={'value': self.load_stopwords})
        
        # stem widgets
        self.stem_choice = pn.widgets.Select(name='Select', options=['Porter', 'Snowball'])
        
        # embedding widgets
        
        self.we_model = pn.widgets.Select(name='Select', options=['SKLearn Count Vectorizer'])

        
    @param.output()
    def output(self):
        return None
    
    
    @param.depends('display_df')
    def df_pane(self):
        return pn.WidgetBox(self.display_df,
                           height = 300,
                           width = 400)
    
    # load text page functions
    #-----------------------------------------------------------------------------------------------------
    def load_df(self, df, event):
        info = io.BytesIO(self.load_file.value)
        if self.header_checkbox.value==True:
            self.df = pd.read_csv(info)
        else:
            self.df = pd.read_csv(info, sep='\n', header = None, names=['text'])
        
        self.display_df = self.df
    
    def load_text_page(self):
        helper_text = (
            "This simple Sentiment Analysis NLP app will allow you to select a few different options " +
            "for some preprocessing steps to prepare your text for testing and training. " +
            "It will then allow you to choose a model to train, the percentage of data to " +
            "preserve for test, while the rest will be used to train the model.  Finally, " +
            "some initial metrics will be displayed to determine how well the model did to predict " +
            "the testing results." +
            " " +
            "Please choose a csv file that contains lines of text to analyze.  This text should " +
            "have a text column as well as a sentiment column.  If there is a header included in the file, " +
            "make sure to check the header checkbox."
        )
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Load Text:'),
                    pn.Column(
                        helper_text,
                         self.header_checkbox,
                         self.load_file
                        ),
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    self.df_pane,
                    
                )
        
        )

    #-----------------------------------------------------------------------------------------------------
    
    # tokenize page options
    #-----------------------------------------------------------------------------------------------------
    def tokenize_option_page(self):
        
        help_text = ("Tokenization will break your text into a list of single articles " +
            "(ex. ['A', 'cat', 'walked', 'into', 'the', 'house', '.']).  Specify a regular " +
            "expression (regex) search pattern to use for splitting the text.")
        
        return pn.Column(
                    pn.pane.Markdown(f'##Tokenize options:'),
                    pn.WidgetBox(help_text, self.search_pattern_input,
                                    height = 300,
                                    width = 300
        
                                )
                )
    
    #-----------------------------------------------------------------------------------------------------
    
    
    # remove stopwords page 
    #-----------------------------------------------------------------------------------------------------
    
    def remove_stopwords_page(self):
        
        help_text = (
            "Stop words are words that do not add any value to the sentiment of the text. " +
            "Removing them may improve your sentiment results.  You may load a list of stop words " +
            "to exclude from your text."
        )
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Load Stopwords:'),
                    pn.WidgetBox(help_text, self.load_words_button,
                                    height = 300,
                                    width = 300
        
                    )
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    pn.WidgetBox(self.stopword_df,
                           height = 300,
                           width = 400)
                    
                )
        )
    
    def load_stopwords(self, stopwords, event):
        info = io.BytesIO(self.load_words_button.value)
        self.stopwords = pd.read_pickle(info)
        self.stopword_df = pd.DataFrame({'stop words': self.stopwords})

    #-----------------------------------------------------------------------------------------------------
    
    # stemming page 
    #-----------------------------------------------------------------------------------------------------
    
    def stemmer_page(self):
        help_text = (
            "Stemming is a normalization step for the words in your text.  Something that is " +
            "plural should probably still be clumped together with a singular version of a word, " +
            "for example.  Stemming will basically remove the ends of words.  Here you can choose " + 
            "between a Porter Stemmer or Snowball Stemmer. Porter is a little less aggressive than " +
            "Snowball, however, Snowball is considered a slight improvement over Porter."
        )
        return pn.Column(
                    pn.pane.Markdown(f'##Stemmer options:'),
                    pn.WidgetBox(help_text, self.stem_choice,
                height = 300,
                width = 300)
                )
    
    #-----------------------------------------------------------------------------------------------------
    
    # embedding page 
    #-----------------------------------------------------------------------------------------------------
    
    def word_embedding_page(self):
        
        help_text = ("Embedding the process of turning words into numerical vectors. " +
                    "There have been several algorithms developed to do this, however, currently in this " +
                    "app, the sklearn count vectorizer is available. This algorithm will return a sparse " +
                    "matrix represention of all the words in your text."
                    )
        
        
        
        return pn.Column(
                    pn.pane.Markdown(f'##Choose embedding model:'),
                    pn.WidgetBox(help_text, self.we_model,
                            height = 300,
                            width = 300
        
                    )
        
                )
    
    #-----------------------------------------------------------------------------------------------------
    
    # *****NEW ADDITION****************************************************************        
    def continue_ready(self, event):

        # Set up for tokenization
        tokenizer = RegexpTokenizer(self.search_pattern_input.value)

        # Set up for stemming
        if self.stem_choice.value == 'Porter':
            stemmer = PorterStemmer() 
        else:
            stemmer = SnowballStemmer()

        # Set up for embedding
        if self.we_model.value == 'SKLearn Count Vectorizer':
            # Create a vectorizer instance
            vectorizer = CountVectorizer(max_features=1000)

        corpus = []
        #loop through each line of data
        for n in range(len(self.display_df)):  
            sentence = self.display_df.iloc[n].text

            #1. Tokenize
            tokens = tokenizer.tokenize(sentence)

            #2. remove stop words
            tokens_no_sw = [word for word in tokens if not word in self.stopwords]

            #3. stem the remaining words
            stem_words = [stemmer.stem(x) for x in tokens_no_sw]

            #Join the words back together as one string and append this string to your corpus.
            corpus.append(' '.join(stem_words))

        X = vectorizer.fit_transform(corpus).toarray()
        labels = self.display_df['sentiment']

        xlist = []
        for n in range(len(X)):
            xlist.append(list(X[n]))
        self.X = X
        self.display_df = pd.DataFrame({'embeddings': xlist, 'sentiment': labels})
        
        #*********************************************************************************
        
    
    def panel(self):
        
        return pn.Column(
            pn.Tabs(
                ('Load Text', self.load_text_page),
                ('Tokenize', self.tokenize_option_page),
                ('Remove Stopwords', self.remove_stopwords_page),
                ('Stem', self.stemmer_page),
                ('Embed', self.word_embedding_page)
                ),
            self.continue_button
        )
        

Now, the first page is done as a stand alone page.  We will have to add some more to it later to fit into our pipeline, but I want to first introduce our second page as there are some details to note regarding this stage.  I am not going to go into as much detail to build it out, but I will say that it is the same layout as the first page.  It will start off displaying a sample of the word embeddings dataframe, then, when the metrics from the train and testing are computed, the display will change to show these results. 

In [ ]:
from app.base import base_page
import panel as pn
import pandas as pd
import param
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np


class trainer(param.Parameterized):
    
    display_df = param.DataFrame(default = pd.DataFrame())
    
    results = param.Boolean(default = False)
    
    X = param.Array(default = None)
    
    result_string = param.String(default = '')

    result_string = param.String('')
    
    def __init__(self, **params):
        super().__init__(**params)
        self.name_of_page = 'Test and Train'
        
        self.test_slider = pn.widgets.IntSlider(name='Test Percentage', start=0, end=100, step=10, value=20)

        self.tt_button = pn.widgets.Button(name='Train and Test', button_type='primary')
        self.tt_button.on_click(self.train_test)
        
        self.tt_model = pn.widgets.Select(name='Select', options=['Random Forrest Classifier'])
        
        
    def train_test(self, event):
        
        #get values from sentiment.
        self.display_df = convert_sentiment_values(self.display_df)
        
        y = self.display_df['label']
        
        #get train test sets
        X_train, X_test, y_train, y_test = train_test_split(self.X, y, test_size = self.test_slider.value/100, random_state = 0)
        
        
        if self.tt_model.value == 'Random Forrest Classifier':
            sentiment_classifier = RandomForestClassifier(n_estimators = 1000, random_state = 0)
            
            sentiment_classifier.fit(X_train, y_train)
            
            y_pred = sentiment_classifier.predict(X_test)
            
        self.y_test = y_test
        self.y_pred = y_pred
        self.analyze()
        
    def analyze(self):
        self.cm = confusion_matrix(self.y_test,self.y_pred)
        self.cr = classification_report(self.y_test,self.y_pred)
        self.acc_score = accuracy_score(self.y_test, self.y_pred)
        
        splits = self.cr.split('\n')
        cml = self.cm.tolist()
        self.result_string = f"""
            ### Classification Report
            <pre>
            {splits[0]}
            {splits[1]}
            {splits[2]}
            {splits[3]}
            {splits[4]}
            {splits[5]}
            {splits[6]}
            {splits[7]}
            {splits[8]}
            </pre>
            ### Confusion Matrix
            <pre>
            {cml[0]}
            {cml[1]}

            </pre>

            ### Accuracy Score
            <pre>
            {round(self.acc_score, 4)}
            </pre
            """
        

        self.results = True 

    def options_page(self, help_text):
        
        return pn.WidgetBox(help_text, self.tt_model,
                            self.test_slider,
                            self.tt_button,
                height = 375,
                width = 300
        
        )
        
    @pn.depends('results')
    def df_pane(self):
        
        if self.results == False:
            self.result_pane = self.display_df
            
        else:
            self.result_pane = pn.pane.Markdown(f"""
                {self.result_string}
                """, width = 500, height = 350)
        
        return pn.WidgetBox(self.result_pane,
                           height = 375,
                           width = 450)
        


    def panel(self):
        
        help_text = (
            "Your text will now be trained and tested using a selected model.  You may " +
            "choose a percentage of your data to reserve for testing, the rest will be used for " +
            "training.  For example, if I reserve 20%, the rest of the 80% will be used for training " +
            "and the 20% will be used to determine how well the trained model does assigning a " +
            "sentiment label to the testing text.  Currently, the only model available is the sklearn " +
            "Random Forrest Classifier model."
        )
        
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Train and Test'),
                    self.options_page(help_text),
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    self.df_pane,
                    
                )
        
        )
    
    
def convert_sentiment_values(df, col = 'sentiment'):
    vals = df['sentiment'].unique()
    df['label'] = 0

    for n in range(len(vals)):
        df['label'] = [n if df[col][x] == vals[n] else df['label'][x] for x in range(len(df[col]))]
        
    return df




    

Something to notice about this stage is that it has a couple of parameters common with stage one, `X` and `display_df`.  The second stage needs the result of stage one, `X`, and a little bit of information stored in the `display_df` dataframe in order to be able to do the training and testing.  The `output` function of the first class can now be updated to return the outputs that we need to pass to stage 2. 

The following will be added to the stage 1 code.

In [ ]:
@param.output('X', 'display_df')
def output(self):
    return self.X, self.display_df

In a panel pipeline, if you need to pass information from one stage to the next, you need to specify the output on one stage, and have the same parameters specified on the next stage.  The paramters on the second stage will essentially consume the output from the stage before.  In our case, we have already set up the training page to have the same parameters specified.  

The final piece needed in place in this panel app is a ready parameter.  This will signal to the pipeline process that we are ready to move from the current stage to the next.  I have chosen to keep it simple and will include a param boolean called `ready` in the PreProcessor class.  The user will click `continue`, the pre-processing will be completed, and then ready will be set to True, which will signal its time to move to stage 2.

In [ ]:
import panel as pn
import param
import pandas as pd

import io

from nltk.stem import (PorterStemmer, SnowballStemmer)
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer

class PreProcessor(param.Parameterized):
    
    # df will be the variable holding the dataframe of text
    df = param.DataFrame()
    # title to display for each tab
    name_of_page = param.String(default = 'Name of page')
    # dataframe to display.
    display_df = param.DataFrame(default = pd.DataFrame())
    # stopword_df is the dataframe containing the stopewords
    stopword_df = param.DataFrame(default = pd.DataFrame())
    
    stopwords = param.List(default = [])
    X = param.Array(default = None)
    
    # *****NEW***********
    ready = param.Boolean(
        default=False,
        doc='trigger for moving to the next page',
        )   
    # *******************
    
    def __init__(self, **params):
        super().__init__(**params)
        
        
        
        # button for the pre-processing page
        self.continue_button = pn.widgets.Button(name='Continue',
                                                 width = 100,
                                                 button_type='primary')

        self.continue_button.on_click(self.continue_ready)
        
        # load text widgets 
        self.header_checkbox = pn.widgets.Checkbox(name='Header included in file')
        self.load_file = pn.widgets.FileInput()
        self.load_file.link(self.df, callbacks={'value': self.load_df})
        self.header_checkbox = pn.widgets.Checkbox(name='Header included in file')
        
        # tokenize widgets
        self.search_pattern_input = pn.widgets.TextInput(name='Search Pattern', value = '\w+', width = 100)
        
        # remove stop words widgets
        self.load_words_button = pn.widgets.FileInput()
        self.load_words_button.link(self.stopwords, callbacks={'value': self.load_stopwords})
        
        # stem widgets
        self.stem_choice = pn.widgets.Select(name='Select', options=['Porter', 'Snowball'])
        
        # embedding widgets
        
        self.we_model = pn.widgets.Select(name='Select', options=['SKLearn Count Vectorizer'])

        
    @param.output('X', 'display_df')
    def output(self):
        return self.X, self.display_df
    
    
    @param.depends('display_df')
    def df_pane(self):
        return pn.WidgetBox(self.display_df,
                           height = 300,
                           width = 400)
    
    # load text page functions
    #-----------------------------------------------------------------------------------------------------
    def load_df(self, df, event):
        info = io.BytesIO(self.load_file.value)
        if self.header_checkbox.value==True:
            self.df = pd.read_csv(info)
        else:
            self.df = pd.read_csv(info, sep='\n', header = None, names=['text'])
        
        self.display_df = self.df
    
    def load_text_page(self):
        helper_text = (
            "This simple Sentiment Analysis NLP app will allow you to select a few different options " +
            "for some preprocessing steps to prepare your text for testing and training. " +
            "It will then allow you to choose a model to train, the percentage of data to " +
            "preserve for test, while the rest will be used to train the model.  Finally, " +
            "some initial metrics will be displayed to determine how well the model did to predict " +
            "the testing results." +
            " " +
            "Please choose a csv file that contains lines of text to analyze.  This text should " +
            "have a text column as well as a sentiment column.  If there is a header included in the file, " +
            "make sure to check the header checkbox."
        )
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Load Text:'),
                    pn.Column(
                        helper_text,
                         self.header_checkbox,
                         self.load_file
                        ),
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    self.df_pane,
                    
                )
        
        )

    #-----------------------------------------------------------------------------------------------------
    
    # tokenize page options
    #-----------------------------------------------------------------------------------------------------
    def tokenize_option_page(self):
        
        help_text = ("Tokenization will break your text into a list of single articles " +
            "(ex. ['A', 'cat', 'walked', 'into', 'the', 'house', '.']).  Specify a regular " +
            "expression (regex) search pattern to use for splitting the text.")
        
        return pn.Column(
                    pn.pane.Markdown(f'##Tokenize options:'),
                    pn.WidgetBox(help_text, self.search_pattern_input,
                                    height = 300,
                                    width = 300
        
                                )
                )
    
    #-----------------------------------------------------------------------------------------------------
    
    
    # remove stopwords page 
    #-----------------------------------------------------------------------------------------------------
    
    def remove_stopwords_page(self):
        
        help_text = (
            "Stop words are words that do not add any value to the sentiment of the text. " +
            "Removing them may improve your sentiment results.  You may load a list of stop words " +
            "to exclude from your text."
        )
        return pn.Row(
                pn.Column(
                    pn.pane.Markdown(f'##Load Stopwords:'),
                    pn.WidgetBox(help_text, self.load_words_button,
                                    height = 300,
                                    width = 300
        
                    )
                ),
                pn.Column(
                    pn.Spacer(height=52),
                    pn.WidgetBox(self.stopword_df,
                           height = 300,
                           width = 400)
                    
                )
        )
    
    def load_stopwords(self, stopwords, event):
        info = io.BytesIO(self.load_words_button.value)
        self.stopwords = pd.read_pickle(info)
        self.stopword_df = pd.DataFrame({'stop words': self.stopwords})

    #-----------------------------------------------------------------------------------------------------
    
    # stemming page 
    #-----------------------------------------------------------------------------------------------------
    
    def stemmer_page(self):
        help_text = (
            "Stemming is a normalization step for the words in your text.  Something that is " +
            "plural should probably still be clumped together with a singular version of a word, " +
            "for example.  Stemming will basically remove the ends of words.  Here you can choose " + 
            "between a Porter Stemmer or Snowball Stemmer. Porter is a little less aggressive than " +
            "Snowball, however, Snowball is considered a slight improvement over Porter."
        )
        return pn.Column(
                    pn.pane.Markdown(f'##Stemmer options:'),
                    pn.WidgetBox(help_text, self.stem_choice,
                height = 300,
                width = 300)
                )
    
    #-----------------------------------------------------------------------------------------------------
    
    # embedding page 
    #-----------------------------------------------------------------------------------------------------
    
    def word_embedding_page(self):
        
        help_text = ("Embedding the process of turning words into numerical vectors. " +
                    "There have been several algorithms developed to do this, however, currently in this " +
                    "app, the sklearn count vectorizer is available. This algorithm will return a sparse " +
                    "matrix represention of all the words in your text."
                    )
        
        
        
        return pn.Column(
                    pn.pane.Markdown(f'##Choose embedding model:'),
                    pn.WidgetBox(help_text, self.we_model,
                            height = 300,
                            width = 300
        
                    )
        
                )
    
    #-----------------------------------------------------------------------------------------------------
          
    def continue_ready(self, event):

        # Set up for tokenization
        tokenizer = RegexpTokenizer(self.search_pattern_input.value)

        # Set up for stemming
        if self.stem_choice.value == 'Porter':
            stemmer = PorterStemmer() 
        else:
            stemmer = SnowballStemmer()

        # Set up for embedding
        if self.we_model.value == 'SKLearn Count Vectorizer':
            # Create a vectorizer instance
            vectorizer = CountVectorizer(max_features=1000)

        corpus = []
        #loop through each line of data
        for n in range(len(self.display_df)):  
            sentence = self.display_df.iloc[n].text

            #1. Tokenize
            tokens = tokenizer.tokenize(sentence)

            #2. remove stop words
            tokens_no_sw = [word for word in tokens if not word in self.stopwords]

            #3. stem the remaining words
            stem_words = [stemmer.stem(x) for x in tokens_no_sw]

            #Join the words back together as one string and append this string to your corpus.
            corpus.append(' '.join(stem_words))

        X = vectorizer.fit_transform(corpus).toarray()
        labels = self.display_df['sentiment']

        xlist = []
        for n in range(len(X)):
            xlist.append(list(X[n]))
        self.X = X
        self.display_df = pd.DataFrame({'embeddings': xlist, 'sentiment': labels})
        
        self.ready = True
    
    def panel(self):
        
        return pn.Column(
            pn.Tabs(
                ('Load Text', self.load_text_page),
                ('Tokenize', self.tokenize_option_page),
                ('Remove Stopwords', self.remove_stopwords_page),
                ('Stem', self.stemmer_page),
                ('Embed', self.word_embedding_page)
                ),
            self.continue_button
        )
        

Finally, we are ready to put our panel pipeline code into place.  For details on how to complete the pipeline, please refer back to this post (reference to pipeline post). 